In [11]:
import pyspark
import random
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.1418848


In [11]:
from pyspark.ml.image import ImageSchema
from sparkdl.image import imageIO as imageIO
from pyspark.sql.functions import lit
from pyspark.ml.image import ImageSchema

img_dir = "datasets/personalities/"

#Read images and Create training & test DataFrames for transfer learning
#jobs_df = imageIO.readImagesWithCustomFn(img_dir + "/jobs", imageIO.PIL_decode).withColumn("label", lit(1))
#zuckerberg_df = imageIO.readImagesWithCustomFn(img_dir + "/zuckerberg",imageIO.PIL_decode).withColumn("label", lit(0))

jobs_df = spark.read.format("image").load(img_dir + "/jobs").withColumn("label", lit(1))
zuckerberg_df = spark.read.format("image").load(img_dir + "/zuckerberg").withColumn("label", lit(0))

display(jobs_df)
display(zuckerberg_df)

jobs_train, jobs_test = jobs_df.randomSplit([0.6, 0.4])
zuckerberg_train, zuckerberg_test = zuckerberg_df.randomSplit([0.6, 0.4])

#dataframe for training a classification model
train_df = jobs_train.unionAll(zuckerberg_train)

#dataframe for testing the classification model
test_df = jobs_test.unionAll(zuckerberg_test)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /Users/Coupang/anaconda3/lib/python3.6/site-packages/IPython/utils/py3compat.py:188 

In [9]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df)

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

df = p_model.transform(test_df)
predictionAndLabels = df.select("label", "prediction", "image")
predictionAndLabels.show()

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Training set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

+-----+----------+--------------------+
|label|prediction|               image|
+-----+----------+--------------------+
|    1|       1.0|[file:///Users/Co...|
|    1|       1.0|[file:///Users/Co...|
|    1|       1.0|[file:///Users/Co...|
|    1|       1.0|[file:///Users/Co...|
|    1|       1.0|[file:///Users/Co...|
|    0|       1.0|[file:///Users/Co...|
|    0|       1.0|[file:///Users/Co...|
|    0|       0.0|[file:///Users/Co...|
|    0|       1.0|[file:///Users/Co...|
|    0|       1.0|[file:///Users/Co...|
|    0|       0.0|[file:///Users/Co...|
|    0|       1.0|[file:///Users/Co...|
|    0|       1.0|[file:///Users/Co...|
|    0|       0.0|[file:///Users/Co...|
|    0|       0.0|[file:///Users/Co...|
|    0|       0.0|[file:///Users/Co...|
|    0|       1.0|[file:///Users/Co...|
+-----+----------+--------------------+

Training set accuracy = 0.5882352941176471
